# <div align="center">8.1 How Learning Differs from Pure Optimization</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

Optimization algorithms used for training of deep models differ from traditional
optimization algorithms in several ways. Machine learning usually acts indirectly.
In most machine learning scenarios, we care about some performance measure
P
, that is defined with respect to the test set and may also be intractable. We
therefore optimize P only indirectly. We reduce a different cost function J(θ) in
the hope that doing so will improve P. This is in contrast to pure optimization,
where minimizing J is a goal in and of itself. Optimization algorithms for training
deep models also typically include some specialization on the specific structure of
machine learning objective functions.

Typically, the cost function can be written as an average over the training set,
such as

<img src='asset/8_1/1.png'>

where L is the per-example loss function, f(x; θ) is the predicted output when
the input is x, pˆdata is the empirical distribution. In the supervised learning case,
y is the target output. Throughout this chapter, we develop the unregularized
supervised case, where the arguments to L are f(x; θ) and y. However, it is trivial
to extend this development, for example, to include θ or x as arguments, or to
exclude y as arguments, in order to develop various forms of regularization or
unsupervised learning.

Equation 8.1 defines an objective function with respect to the training set. We
would usually prefer to minimize the corresponding objective function where the
expectation is taken across the data generating distribution pdata rather than just
over the finite training set:

<img src='asset/8_1/2.png'>

## <div align="center">8.1.1 Empirical Risk Minimization</div>
---------------------------------------------------------------------

The goal of a machine learning algorithm is to reduce the expected generalization
error given by equation 8.2. This quantity is known as the risk. We emphasize here
that the expectation is taken over the true underlying distribution pdata. If we knew
the true distribution pdata(x, y), risk minimization would be an optimization task solvable by an optimization algorithm. However, when we do not know pdata(x, y)
but only have a training set of samples, we have a machine learning problem.
The simplest way to convert a machine learning problem back into an optimization problem is to minimize the expected loss on the training set. This
means replacing the true distribution p(x, y) with the empirical distribution pˆ(x, y)
defined by the training set. We now minimize the empirical risk

<img src='asset/8_1/3.png'>

where m is the number of training examples.

The training process based on minimizing this average training error is known
as empirical risk minimization. In this setting, machine learning is still very
similar to straightforward optimization. Rather than optimizing the risk directly,
we optimize the empirical risk, and hope that the risk decreases significantly as
well. A variety of theoretical results establish conditions under which the true risk
can be expected to decrease by various amounts.

However, empirical risk minimization is prone to overfitting. Models with
high capacity can simply memorize the training set. In many cases, empirical
risk minimization is not really feasible. The most effective modern optimization
algorithms are based on gradient descent, but many useful loss functions, such
as 0-1 loss, have no useful derivatives (the derivative is either zero or undefined
everywhere). These two problems mean that, in the context of deep learning, we
rarely use empirical risk minimization. Instead, we must use a slightly different
approach, in which the quantity that we actually optimize is even more different
from the quantity that we truly want to optimize.

## <div align="center">8.1.2 Surrogate Loss Functions and Early Stopping</div>
---------------------------------------------------------------------

Sometimes, the loss function we actually care about (say classification error) is not
one that can be optimized efficiently. For example, exactly minimizing expected 0-1
loss is typically intractable (exponential in the input dimension), even for a linear
classifier (Marcotte and Savard, 1992). In such situations, one typically optimizes
a surrogate loss function instead, which acts as a proxy but has advantages.
For example, the negative log-likelihood of the correct class is typically used as a
surrogate for the 0-1 loss. The negative log-likelihood allows the model to estimate
the conditional probability of the classes, given the input, and if the model can
do that well, then it can pick the classes that yield the least classification error in
expectation.

In some cases, a surrogate loss function actually results in being able to learn
more. For example, the test set 0-1 loss often continues to decrease for a long
time after the training set 0-1 loss has reached zero, when training using the
log-likelihood surrogate. This is because even when the expected 0-1 loss is zero,
one can improve the robustness of the classifier by further pushing the classes apart
from each other, obtaining a more confident and reliable classifier, thus extracting
more information from the training data than would have been possible by simply
minimizing the average 0-1 loss on the training set

A very important difference between optimization in general and optimization
as we use it for training algorithms is that training algorithms do not usually halt
at a local minimum. Instead, a machine learning algorithm usually minimizes
a surrogate loss function but halts when a convergence criterion based on early
stopping is satisfied. Typically the early stopping criterion is based
on the true underlying loss function, such as 0-1 loss measured on a validation set,
and is designed to cause the algorithm to halt whenever overfitting begins to occur.
Training often halts while the surrogate loss function still has large derivatives,
which is very different from the pure optimization setting, where an optimization
algorithm is considered to have converged when the gradient becomes very small.

## <div align="center">8.1.3 Batch and Minibatch Algorithms</div>
---------------------------------------------------------------------

One aspect of machine learning algorithms that separates them from general
optimization algorithms is that the objective function usually decomposes as a sum
over the training examples. Optimization algorithms for machine learning typically
compute each update to the parameters based on an expected value of the cost
function estimated using only a subset of the terms of the full cost function.

For example, maximum likelihood estimation problems, when viewed in log
space, decompose into a sum over each example:

<img src='asset/8_1/4.png'>

Maximizing this sum is equivalent to maximizing the expectation over the
empirical distribution defined by the training set:

<img src='asset/8_1/5.png'>

Most of the properties of the objective function J used by most of our optimization algorithms are also expectations over the training set. For example, the most commonly used property is the gradient:

<img src='asset/8_1/6.png'>

Computing this expectation exactly is very expensive because it requires
evaluating the model on every example in the entire dataset. In practice, we can
compute these expectations by randomly sampling a small number of examples
from the dataset, then taking the average over only those examples

Recall that the standard error of the mean estimated from n
samples is given by σ/√n, where σ is the true standard deviation of the value of
the samples. The denominator of √n shows that there are less than linear returns
to using more examples to estimate the gradient. Compare two hypothetical
estimates of the gradient, one based on 100 examples and another based on 10,000
examples. The latter requires 100 times more computation than the former, but
reduces the standard error of the mean only by a factor of 10. Most optimization
algorithms converge much faster (in terms of total computation, not in terms of
number of updates) if they are allowed to rapidly compute approximate estimates
of the gradient rather than slowly computing the exact gradient

Another consideration motivating statistical estimation of the gradient from a
small number of samples is redundancy in the training set. In the worst case, all
m samples in the training set could be identical copies of each other. A samplingbased estimate of the gradient could compute the correct gradient with a single
sample, using m times less computation than the naive approach. In practice, we
are unlikely to truly encounter this worst-case situation, but we may find large
numbers of examples that all make very similar contributions to the gradient.

Optimization algorithms that use the entire training set are called ***batch or
deterministic gradient methods***, because they process all of the training examples
simultaneously in a large batch. This terminology can be somewhat confusing
because the word “batch” is also often used to describe the minibatch used by
minibatch stochastic gradient descent. Typically the term “batch gradient descent”
implies the use of the full training set, while the use of the term “batch” to describe
a group of examples does not. For example, it is very common to use the term
“batch size” to describe the size of a minibatch.

Optimization algorithms that use only a single example at a time are sometimes
called stochastic or sometimes online methods. The term online is usually
reserved for the case where the examples are drawn from a stream of continually
created examples rather than from a fixed-size training set over which several
passes are made.

Most algorithms used for deep learning fall somewhere in between, using more than one but less than all of the training examples. These were traditionally called
minibatch or minibatch stochastic methods and it is now common to simply
call them ***stochastic methods***.

The canonical example of a stochastic method is stochastic gradient descent,
presented in detail in section 8.3.1.

Minibatch sizes are generally driven by the following factors:

*  Larger batches provide a more accurate estimate of the gradient, but with less than linear returns.


* Multicore architectures are usually underutilized by extremely small batches. This motivates using some absolute minimum batch size, below which there is no reduction in the time to process a minibatch.


* If all examples in the batch are to be processed in parallel (as is typically the case), then the amount of memory scales with the batch size. For many hardware setups this is the limiting factor in batch size.


* Some kinds of hardware achieve better runtime with specific sizes of arrays. Especially when using GPUs, it is common for power of 2 batch sizes to offer better runtime. Typical power of 2 batch sizes range from 32 to 256, with 16 sometimes being attempted for large models.


*  Small batches can offer a regularizing effect (Wilson and Martinez, 2003), perhaps due to the noise they add to the learning process. Generalization error is often best for a batch size of 1. Training with such a small batch size might require a small learning rate to maintain stability due to the high variance in the estimate of the gradient. The total runtime can be very high due to the need to make more steps, both because of the reduced learning rate and because it takes more steps to observe the entire training set.

Different kinds of algorithms use different kinds of information from the minibatch in different ways. Some algorithms are more sensitive to sampling error than
others, either because they use information that is difficult to estimate accurately
with few samples, or because they use information in ways that amplify sampling
errors more. Methods that compute updates based only on the gradient g are
usually relatively robust and can handle smaller batch sizes like 100. Second-order
methods, which use also the Hessian matrix H and compute updates such as
$H^{−1}g$, typically require much larger batch sizes like 10,000. These large batch
sizes are required to minimize fluctuations in the estimates of $H^{−1}g$. Suppose
that H is estimated perfectly but has a poor condition number. Multiplication by H or its inverse amplifies pre-existing errors, in this case, estimation errors in g.
Very small changes in the estimate of g can thus cause large changes in the update
$H^{−1}g$, even if H were estimated perfectly. Of course, H will be estimated only
approximately, so the update $H^{−1}g$ will contain even more error than we would
predict from applying a poorly conditioned operation to the estimate of g.

It is also crucial that the minibatches be selected randomly. Computing an
unbiased estimate of the expected gradient from a set of samples requires that those
samples be independent. We also wish for two subsequent gradient estimates to be
independent from each other, so two subsequent minibatches of examples should
also be independent from each other. Many datasets are most naturally arranged
in a way where successive examples are highly correlated. For example, we might
have a dataset of medical data with a long list of blood sample test results. This
list might be arranged so that first we have five blood samples taken at different
times from the first patient, then we have three blood samples taken from the
second patient, then the blood samples from the third patient, and so on. If we
were to draw examples in order from this list, then each of our minibatches would
be extremely biased, because it would represent primarily one patient out of the
many patients in the dataset. In cases such as these where the order of the dataset
holds some significance, it is necessary to shuffle the examples before selecting
minibatches. For very large datasets, for example datasets containing billions of
examples in a data center, it can be impractical to sample examples truly uniformly
at random every time we want to construct a minibatch. Fortunately, in practice
it is usually sufficient to shuffle the order of the dataset once and then store it in
shuffled fashion. This will impose a fixed set of possible minibatches of consecutive
examples that all models trained thereafter will use, and each individual model
will be forced to reuse this ordering every time it passes through the training
data. However, this deviation from true random selection does not seem to have a
significant detrimental effect. Failing to ever shuffle the examples in any way can
seriously reduce the effectiveness of the algorithm.

Many optimization problems in machine learning decompose over examples
well enough that we can compute entire separate updates over different examples
in parallel. In other words, we can compute the update that minimizes J(X) for
one minibatch of examples X at the same time that we compute the update for
several other minibatches.

An interesting motivation for minibatch stochastic gradient descent is that it
follows the gradient of the true generalization error (equation 8.2) so long as no
examples are repeated. Most implementations of minibatch stochastic gradient descent shuffle the dataset once and then pass through it multiple times. On the
first pass, each minibatch is used to compute an unbiased estimate of the true
generalization error. On the second pass, the estimate becomes biased because it is
formed by re-sampling values that have already been used, rather than obtaining
new fair samples from the data generating distribution.

The fact that stochastic gradient descent minimizes generalization error is
easiest to see in the online learning case, where examples or minibatches are drawn
from a stream of data. In other words, instead of receiving a fixed-size training
set, the learner is similar to a living being who sees a new example at each instant,
with every example (x, y) coming from the data generating distribution $p_{data(x, y)}$.
In this scenario, examples are never repeated; every experience is a fair sample
from $p_{data}$.

The equivalence is easiest to derive when both x and y are discrete. In this
case, the generalization error (equation 8.2) can be written as a sum

<img src='asset/8_1/7.png'>

with the exact gradient

<img src='asset/8_1/8.png'>

We have already seen the same fact demonstrated for the log-likelihood in equation 8.5 and equation 8.6; we observe now that this holds for other functions L
besides the likelihood. A similar result can be derived when x and y are continuous,
under mild assumptions regarding pdata and L.

Hence, we can obtain an unbiased estimator of the exact gradient of the
generalization error by sampling a minibatch of examples {x(1), . . . x(m)} with corresponding targets y(i) from the data generating distribution pdata, and computing
the gradient of the loss with respect to the parameters for that minibatch:

<img src='asset/8_1/9.png'>

Updating θ in the direction of gˆ performs SGD on the generalization error.

Of course, this interpretation only applies when examples are not reused.
Nonetheless, it is usually best to make several passes through the training set,
unless the training set is extremely large. When multiple such epochs are used,
only the first epoch follows the unbiased gradient of the generalization error, but of course, the additional epochs usually provide enough benefit due to decreased
training error to offset the harm they cause by increasing the gap between training
error and test error.

With some datasets growing rapidly in size, faster than computing power, it
is becoming more common for machine learning applications to use each training
example only once or even to make an incomplete pass through the training
set. When using an extremely large training set, overfitting is not an issue, so
underfitting and computational efficiency become the predominant concerns.